# 🗺️ #30DayMapChallenge 2024 - Day 13: A New Tool 🔧

## **Exploring Climate Through Visualization: Okinawa's Winds and Temperature** 🌦️🌀

### 1. A New Tool in Action! 🎯

For Day 13, I decided to step out of my comfort zone and dive into climate data visualization. As a GIS and Python instructor, this marks my first experience working with weather data. Inspired by the beauty of Japan's subtropical Okinawa, I created an interactive map that merges scientific precision with storytelling.


### 2. Why Okinawa? 🏝️

Okinawa, the southernmost prefecture of Japan, stands apart geographically and culturally. Historically, it was the heart of the Ryukyu Kingdom, thriving as a bridge between Japan, China, and Southeast Asia. Its location makes it a hotspot for unique climate patterns, from typhoons to balmy tropical breezes.

#### **Quick Facts about Okinawa:**
- 🌏 **Location**: Between Taiwan and mainland Japan
- 🌀 **Weather**: Tropical climate, often influenced by the Pacific typhoon season
- 📜 **History**: A culturally distinct region with influences from its Ryukyu heritage
- 🐠 **Environment**: Coral reefs, turquoise seas, and rich biodiversity


### 3. Map Design Elements 🎨

To capture Okinawa's dynamic weather, I used Python with the Folium library. This visualization highlights:
1. **Temperature Variations**: A heatmap layer shows temperature gradients across the island.
2. **Wind Dynamics**: Arrows represent wind direction and intensity.
3. **Weather Highlights**: Key weather conditions are overlaid as markers.

#### Visual Hierarchy
- **Temperature**: Smooth gradients transitioning from cool blues to fiery reds
- **Wind Arrows**: Direction and speed visualized through dynamic scaling
- **Interactive Markers**: Popups display current weather details at specific locations


### 4. Data Pipeline 🔧

#### **Source Data**
- OpenWeatherMap API for real-time weather conditions
- Geographic data from OpenStreetMap for Okinawa's boundaries

#### **Processing Steps**
1. **Grid Generation**: Dividing Okinawa into a 10x15 point grid for precise weather readings.
2. **Data Retrieval**: Fetching temperature, wind speed, and weather details from the API.
3. **Visualization**: Using Folium layers to create an engaging and interactive map.

#### **Technical Highlights**
- **Temperature Layer**: Heatmaps with dynamic radius and color intensity.
- **Wind Layer**: Scalable arrows styled with SVG icons.
- **Weather Popups**: Display real-time conditions with user-friendly tooltips.


### 5. Japanese Context 🌸

**Why does Okinawa feel so distant?**
Okinawa’s unique location mirrors its distinct history. As the Ryukyu Kingdom, it was an independent hub for maritime trade until annexation by Japan in the 19th century. Even today, its tropical climate and traditions make it feel like a world apart from the rest of Japan.


### 6. Where to Insert the Code? 💻

Place your Python code in the **"Technical Highlights"** section above. Here's a placeholder for your script:


In [1]:
!pip install folium -q

In [10]:
import folium
from folium import plugins
import numpy as np
import requests
import math
from datetime import datetime
import branca.colormap as cm

def get_weather_data(lat_min, lat_max, lon_min, lon_max, api_key, grid_size=20):
    """
    Gets weather data for a rectangular area using OpenWeatherMap
    """
    if not api_key:
        return _get_simulated_weather_data(lat_min, lat_max, lon_min, lon_max, grid_size)

    lats = np.linspace(lat_min, lat_max, grid_size)
    lons = np.linspace(lon_min, lon_max, int(grid_size * 1.5))
    data_points = []

    # Get predominant wind data for the region
    center_lat = (lat_min + lat_max) / 2
    center_lon = (lon_min + lon_max) / 2
    base_url = "http://api.openweathermap.org/data/2.5/weather"

    try:
        response = requests.get(f"{base_url}?lat={center_lat}&lon={center_lon}&appid={api_key}&units=metric")
        response.raise_for_status()
        base_weather = response.json()
        base_wind_speed = base_weather['wind']['speed']
        base_wind_dir = base_weather['wind']['deg']

        # Create realistic wind variation around base value
        for lat in lats:
            for lon in lons:
                try:
                    response = requests.get(
                        f"{base_url}?lat={lat}&lon={lon}&appid={api_key}&units=metric"
                    )
                    response.raise_for_status()
                    weather_data = response.json()

                    # Add smooth variation to base wind
                    wind_variation_speed = np.random.normal(0, base_wind_speed * 0.1)
                    wind_variation_dir = np.random.normal(0, 10)  # ±10 degrees variation

                    data_points.append({
                        'lat': lat,
                        'lon': lon,
                        'temp': weather_data['main']['temp'],
                        'humidity': weather_data['main']['humidity'],
                        'wind_speed': base_wind_speed + wind_variation_speed,
                        'wind_dir': (base_wind_dir + wind_variation_dir) % 360,
                        'weather_main': weather_data['weather'][0]['main'],
                        'weather_description': weather_data['weather'][0]['description'],
                        'weather_icon': weather_data['weather'][0]['icon']
                    })
                except requests.RequestException as e:
                    print(f"Error getting data for point ({lat}, {lon}): {e}")
                    continue

    except requests.RequestException as e:
        print(f"Error getting base weather data: {e}")
        return _get_simulated_weather_data(lat_min, lat_max, lon_min, lon_max, grid_size)

    return data_points

def _get_simulated_weather_data(lat_min, lat_max, lon_min, lon_max, grid_size):
    """
    Generates more realistic simulated weather data for testing
    """
    lats = np.linspace(lat_min, lat_max, grid_size)
    lons = np.linspace(lon_min, lon_max, int(grid_size * 1.5))

    # Generate more realistic wind pattern
    base_wind_speed = 5 + np.random.normal(0, 1)
    base_wind_dir = np.random.randint(0, 360)

    data_points = []
    weather_conditions = [
        {'main': 'Clear', 'description': 'clear sky', 'icon': '01d'},
        {'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'},
        {'main': 'Rain', 'description': 'light rain', 'icon': '10d'}
    ]

    # Choose predominant condition for more realism
    main_condition = np.random.choice(weather_conditions)

    for lat in lats:
        for lon in lons:
            # Smooth wind variation
            wind_variation_speed = np.random.normal(0, base_wind_speed * 0.1)
            wind_variation_dir = np.random.normal(0, 10)

            # 80% chance to keep predominant condition
            weather = main_condition if np.random.random() < 0.8 else np.random.choice(weather_conditions)

            data_points.append({
                'lat': lat,
                'lon': lon,
                'temp': 25 + np.random.normal(0, 2),
                'humidity': 70 + np.random.normal(0, 5),
                'wind_speed': base_wind_speed + wind_variation_speed,
                'wind_dir': (base_wind_dir + wind_variation_dir) % 360,
                'weather_main': weather['main'],
                'weather_description': weather['description'],
                'weather_icon': weather['icon']
            })

    return data_points

def get_weather_icon_html(icon_code):
    """
    Returns HTML for weather icon based on OpenWeather code
    """
    weather_icons = {
        '01d': '☀️',  # clear sky day
        '01n': '🌙',  # clear sky night
        '02d': '⛅️',  # few clouds day
        '02n': '☁️',  # few clouds night
        '03d': '☁️',  # scattered clouds
        '03n': '☁️',
        '04d': '☁️',  # broken clouds
        '04n': '☁️',
        '09d': '🌧️',  # shower rain
        '09n': '🌧️',
        '10d': '🌦️',  # rain
        '10n': '🌧️',
        '11d': '⛈️',  # thunderstorm
        '11n': '⛈️',
        '13d': '🌨️',  # snow
        '13n': '🌨️',
        '50d': '🌫️',  # mist
        '50n': '🌫️'
    }
    return weather_icons.get(icon_code, '☁️')

def create_wind_arrow_icon(direction, speed):
    """
    Creates enhanced wind arrow icon
    """
    size = min(20, 8 + (speed * 0.8))

    # Adjust color based on wind speed
    if speed < 5:
        color = "#1a4b8c"  # Darker light blue
    elif speed < 10:
        color = "#143870"  # Darker medium blue
    elif speed < 15:
        color = "#0c2654"  # Darker strong blue
    else:
        color = "#061333"  # Even darker blue for very strong wind

    opacity = min(1.0, 0.6 + (speed / 20))

    svg = f'''
        <svg width="{size*2}" height="{size*2}" viewBox="0 0 100 100" xmlns="http://www.w3.org/2000/svg">
            <polygon points="50,10 35,90 50,70 65,90"
                     transform="rotate({direction}, 50, 50)"
                     fill="{color}"
                     stroke="white"
                     stroke-width="1"
                     opacity="{opacity}"/>
        </svg>
    '''
    return svg

def create_interactive_weather_map(lat_min, lat_max, lon_min, lon_max, api_key, center_lat, center_lon):
    """
    Creates interactive map with enhanced features
    """
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=11,
        tiles='OpenStreetMap',
        attr='© OpenStreetMap contributors',
        zoom_control=False  # Remove zoom control buttons
    )

    # Get weather data
    data_points = get_weather_data(lat_min, lat_max, lon_min, lon_max, api_key)

    # Create temperature layer with enhanced gradient
    heat_data = [[point['lat'], point['lon'], point['temp']] for point in data_points]
    temp_layer = plugins.HeatMap(
        heat_data,
        name='Temperature',
        min_opacity=0.2,
        radius=15,
        blur=20,
        gradient={
            0.2: '#313695',  # dark blue (cold)
            0.4: '#4575b4',
            0.6: '#74add1',
            0.8: '#fed976',  # yellow
            1.0: '#bd0026'   # red (hot)
        }
    )

    # Create wind arrows layer
    wind_layer = folium.FeatureGroup(name='Wind')

    for point in data_points:
        icon_html = create_wind_arrow_icon(point['wind_dir'], point['wind_speed'])
        icon = folium.DivIcon(html=icon_html)

        folium.Marker(
            location=[point['lat'], point['lon']],
            icon=icon,
            popup=folium.Popup(
                f"""
                <div style="font-family: Arial, sans-serif;">
                <b>Weather Conditions</b><br>
                Temperature: {point['temp']:.1f}°C<br>
                Wind: {point['wind_speed']:.1f} m/s<br>
                Direction: {point['wind_dir']}°<br>
                Conditions: {point['weather_description']}
                </div>
                """,
                max_width=200
            )
        ).add_to(wind_layer)

    # Add layers to map
    temp_layer.add_to(m)
    wind_layer.add_to(m)

    # Add marker for central location
    folium.CircleMarker(
        location=[center_lat, center_lon],
        radius=8,
        color='red',
        fill=True,
        popup='Naha'
    ).add_to(m)

    # Add controls (except zoom)
    plugins.MiniMap(toggle_display=True).add_to(m)
    folium.LayerControl().add_to(m)

    # Get center point data for info panel
    center_point = min(data_points, key=lambda x:
        math.sqrt((x['lat'] - center_lat)**2 + (x['lon'] - center_lon)**2))

    # Enhanced weather info panel
    weather_html = f'''
        <div style="position: fixed;
                    top: 20px;
                    left: 60px;
                    background-color: rgba(255, 255, 255, 0.9);
                    padding: 15px;
                    border-radius: 8px;
                    box-shadow: 0 0 15px rgba(0,0,0,0.2);
                    z-index: 1000;
                    font-family: Arial, sans-serif;">
            <div style="font-size: 32px;">{get_weather_icon_html(center_point['weather_icon'])}</div>
            <div style="font-size: 18px; font-weight: bold;">Naha</div>
            <div style="font-size: 16px;">{center_point['temp']:.1f}°C</div>
            <div style="font-size: 14px;">Wind: {center_point['wind_speed']:.1f} m/s</div>
            <div style="font-size: 14px;">{center_point['weather_description'].title()}</div>
            <div style="font-size: 12px; color: #666;">Humidity: {center_point['humidity']}%</div>
        </div>
    '''
    m.get_root().html.add_child(folium.Element(weather_html))

    return m

# Parameters for rectangular area (increased area)
center_lat, center_lon = 26.2124, 127.6809  # Naha
lat_range = 0.3  # Increased from 0.1
lon_range = 0.6  # Increased from 0.2
lat_min = center_lat - lat_range/2
lat_max = center_lat + lat_range/2
lon_min = center_lon - lon_range/2
lon_max = center_lon + lon_range/2

# API key (replace with your OpenWeather API key)
api_key = "YOUR_API_CODE_HERE"

try:
    print("Creating weather map...")
    map_enhanced = create_interactive_weather_map(
        lat_min, lat_max, lon_min, lon_max,
        api_key, center_lat, center_lon
    )
    map_enhanced.save('weather_map_enhanced.html')
    print("Map created successfully!")
except Exception as e:
    print(f"Error creating map: {str(e)}")

Creating weather map...
Map created successfully!


### 7. Reflections 🌟

This map was more than a technical exercise—it was a journey through uncharted territory. By blending data, culture, and visuals, I discovered how GIS can bridge the gap between climate science and storytelling.

#### **Takeaways**
- 🌏 Exploring climate data can uncover stories hidden in plain sight.
- 🔧 Trying a new tool is less about the tool itself and more about what you can create with it.
- 🌸 Okinawa reminds us of the rich interplay between geography and culture.

### 8. Resources 📚
- [OpenWeatherMap API](https://openweathermap.org/api)
- [Folium Documentation](https://python-visualization.github.io/folium/)
- [Okinawa Travel Guide](https://www.visitokinawa.jp/)

In [11]:
#@title Follow Me!
%%html

<div style="background: linear-gradient(90deg, #1a1a1a, #2d2d2d); color: white; padding: 2rem; border-radius: 10px; font-family: 'Roboto', sans-serif; margin-top: 2rem;">
    <h2 style="text-align: center; margin-bottom: 1rem;">
        <span style="display: block; color: white; font-size: 2.7rem; margin-bottom: 0.5rem;">ありがとうございます！</span>
        <span style="display: block; color: white; font-size: 1.8rem;">🌍 Thank you for making it this far! 🎌</span>
    </h2>

    <p style="font-size: 1.2rem; margin-bottom: 1.5rem;">If you're interested in the world of geoinformatics and want to keep learning, connect with me! | 一緒に学びましょう！</p>

    <div style="display: flex; flex-wrap: wrap; gap: 1rem; margin-bottom: 1rem;">
        <a href="https://x.com/GastonEchenique" target="_blank" style="text-decoration: none; color: white; display: flex; align-items: center; background: #1DA1F2; padding: 0.5rem 1rem; border-radius: 5px; transition: transform 0.2s;">
            <svg style="width: 24px; height: 24px; margin-right: 0.5rem;" viewBox="0 0 24 24" fill="currentColor">
                <path d="M18.244 2.25h3.308l-7.227 8.26 8.502 11.24H16.17l-5.214-6.817L4.99 21.75H1.68l7.73-8.835L1.254 2.25H8.08l4.713 6.231zm-1.161 17.52h1.833L7.084 4.126H5.117z"/>
            </svg>
            <span>@GastonEchenique</span>
        </a>

        <a href="https://www.linkedin.com/in/gaston-echenique/" target="_blank" style="text-decoration: none; color: white; display: flex; align-items: center; background: #0A66C2; padding: 0.5rem 1rem; border-radius: 5px; transition: transform 0.2s;">
            <svg style="width: 24px; height: 24px; margin-right: 0.5rem;" viewBox="0 0 24 24" fill="currentColor">
                <path d="M20.447 20.452h-3.554v-5.569c0-1.328-.027-3.037-1.852-3.037-1.853 0-2.136 1.445-2.136 2.939v5.667H9.351V9h3.414v1.561h.046c.477-.9 1.637-1.85 3.37-1.85 3.601 0 4.267 2.37 4.267 5.455v6.286zM5.337 7.433c-1.144 0-2.063-.926-2.063-2.065 0-1.138.92-2.063 2.063-2.063 1.14 0 2.064.925 2.064 2.063 0 1.139-.925 2.065-2.064 2.065zm1.782 13.019H3.555V9h3.564v11.452zM22.225 0H1.771C.792 0 0 .774 0 1.729v20.542C0 23.227.792 24 1.771 24h20.451C23.2 24 24 23.227 24 22.271V1.729C24 .774 23.2 0 22.222 0h.003z"/>
            </svg>
            <span>Gastón Echenique</span>
        </a>

        <a href="https://github.com/oechenique" target="_blank" style="text-decoration: none; color: white; display: flex; align-items: center; background: #333; padding: 0.5rem 1rem; border-radius: 5px; transition: transform 0.2s;">
            <svg style="width: 24px; height: 24px; margin-right: 0.5rem;" viewBox="0 0 24 24" fill="currentColor">
                <path d="M12 0c-6.626 0-12 5.373-12 12 0 5.302 3.438 9.8 8.207 11.387.599.111.793-.261.793-.577v-2.234c-3.338.726-4.033-1.416-4.033-1.416-.546-1.387-1.333-1.756-1.333-1.756-1.089-.745.083-.729.083-.729 1.205.084 1.839 1.237 1.839 1.237 1.07 1.834 2.807 1.304 3.492.997.107-.775.418-1.305.762-1.604-2.665-.305-5.467-1.334-5.467-5.931 0-1.311.469-2.381 1.236-3.221-.124-.303-.535-1.524.117-3.176 0 0 1.008-.322 3.301 1.23.957-.266 1.983-.399 3.003-.404 1.02.005 2.047.138 3.006.404 2.291-1.552 3.297-1.23 3.297-1.23.653 1.653.242 2.874.118 3.176.77.84 1.235 1.911 1.235 3.221 0 4.609-2.807 5.624-5.479 5.921.43.372.823 1.102.823 2.222v3.293c0 .319.192.694.801.576 4.765-1.589 8.199-6.086 8.199-11.386 0-6.627-5.373-12-12-12z"/>
            </svg>
            <span>oechenique</span>
        </a>

        <a href="https://oechenique.github.io/geoanalytics/" target="_blank" style="text-decoration: none; color: white; display: flex; align-items: center; background: #2ecc71; padding: 0.5rem 1rem; border-radius: 5px; transition: transform 0.2s;">
            <svg style="width: 24px; height: 24px; margin-right: 0.5rem;" viewBox="0 0 24 24" fill="currentColor">
                <path d="M12 0c-6.627 0-12 5.373-12 12s5.373 12 12 12 12-5.373 12-12-5.373-12-12-12zm1 16.057v-3.057h2.994c-.059 1.143-.212 2.24-.456 3.279-.823-.12-1.674-.188-2.538-.222zm1.957 2.162c-.499 1.33-1.159 2.497-1.957 3.456v-3.62c.666.028 1.319.081 1.957.164zm-1.957-7.219v-3.015c.868-.034 1.721-.103 2.548-.224.238 1.027.389 2.111.446 3.239h-2.994zm0-5.014v-3.661c.806.969 1.471 2.15 1.971 3.496-.642.084-1.3.137-1.971.165zm2.703-3.267c1.237.496 2.354 1.228 3.29 2.146-.642.234-1.311.442-2.019.607-.344-.992-.775-1.91-1.271-2.753zm-7.241 13.56c-.244-1.039-.398-2.136-.456-3.279h2.994v3.057c-.865.034-1.714.102-2.538.222zm2.538 1.776v3.62c-.798-.959-1.458-2.126-1.957-3.456.638-.083 1.291-.136 1.957-.164zm-2.994-7.055c.057-1.128.207-2.212.446-3.239.827.121 1.68.19 2.548.224v3.015h-2.994zm1.024-5.179c.5-1.346 1.165-2.527 1.97-3.496v3.661c-.671-.028-1.329-.081-1.97-.165zm-2.005-.35c-.708-.165-1.377-.373-2.018-.607.937-.918 2.053-1.65 3.29-2.146-.496.844-.927 1.762-1.272 2.753zm-.549 1.918c-.264 1.151-.434 2.36-.492 3.611h-3.933c.165-1.658.739-3.197 1.617-4.518.88.361 1.816.67 2.808.907zm.009 9.262c-.988.236-1.92.542-2.797.9-.89-1.328-1.471-2.879-1.637-4.551h3.934c.058 1.265.231 2.488.5 3.651zm.553 1.917c.342.976.768 1.881 1.257 2.712-1.223-.49-2.326-1.211-3.256-2.115.636-.229 1.299-.435 1.999-.597zm9.924 0c.7.163 1.362.367 1.999.597-.931.903-2.034 1.625-3.257 2.116.489-.832.915-1.737 1.258-2.713zm.553-1.917c.27-1.163.442-2.386.501-3.651h3.934c-.167 1.672-.748 3.223-1.638 4.551-.877-.358-1.81-.664-2.797-.9zm.501-5.651c-.058-1.251-.229-2.46-.492-3.611.992-.237 1.929-.546 2.809-.907.877 1.321 1.451 2.86 1.616 4.518h-3.933z"/>
            </svg>
            <span>GeoAnalytics</span>
        </a>

        <a href="https://discord.com/users/gastonechenique" target="_blank" style="text-decoration: none; color: white; display: flex; align-items: center; background: #5865F2; padding: 0.5rem 1rem; border-radius: 5px; transition: transform 0.2s;">
            <svg style="width: 24px; height: 24px; margin-right: 0.5rem;" viewBox="0 0 24 24" fill="currentColor">
                <path d="M20.317 4.37a19.791 19.791 0 0 0-4.885-1.515a.074.074 0 0 0-.079.037c-.21.375-.444.864-.608 1.25a18.27 18.27 0 0 0-5.487 0a12.64 12.64 0 0 0-.617-1.25a.077.077 0 0 0-.079-.037A19.736 19.736 0 0 0 3.677 4.37a.07.07 0 0 0-.032.027C.533 9.046-.32 13.58.099 18.057a.082.082 0 0 0 .031.057a19.9 19.9 0 0 0 5.993 3.03a.078.078 0 0 0 .084-.028a14.09 14.09 0 0 0 1.226-1.994a.076.076 0 0 0-.041-.106a13.107 13.107 0 0 1-1.872-.892a.077.077 0 0 1-.008-.128a10.2 10.2 0 0 0 .372-.292a.074.074 0 0 1 .077-.01c3.928 1.793 8.18 1.793 12.062 0a.074.074 0 0 1 .078.01c.12.098.246.198.373.292a.077.077 0 0 1-.006.127a12.299 12.299 0 0 1-1.873.892a.077.077 0 0 0-.041.107c.36.698.772 1.362 1.225 1.993a.076.076 0 0 0 .084.028a19.839 19.839 0 0 0 6.002-3.03a.077.077 0 0 0 .032-.054c.5-5.177-.838-9.674-3.549-13.66a.061.061 0 0 0-.031-.03zM8.02 15.33c-1.183 0-2.157-1.085-2.157-2.419c0-1.333.956-2.419 2.157-2.419c1.21 0 2.176 1.096 2.157 2.42c0 1.333-.956 2.418-2.157 2.418zm7.975 0c-1.183 0-2.157-1.085-2.157-2.419c0-1.333.955-2.419 2.157-2.419c1.21 0 2.176 1.096 2.157 2.42c0 1.333-.946 2.418-2.157 2.418z"/>
            </svg>
            <span>Gaston | ガストン</span>
        </a>
    </div>

    <p style="font-size: 1rem; color: #888;">💡 頑張りましょう！Don't miss more content about GIS, Data Science and Geoinformatics!</p>

    <div style="margin-top: 1rem; padding-top: 1rem; border-top: 1px solid #444; font-size: 0.9rem; color: #666;">
        Made with 🗺️ & 💻 by Gaston Echenique | エチェニケ・ガストン
    </div>
</div>